In [404]:
with open('day_05.txt') as f:
    data = [int(token) for token in f.readlines()[0].strip().split(',')]

In [405]:
# Opcode IDs
ADD = 1
MULTIPLY = 2
INPUT = 3
OUTPUT = 4
JUMP_IF_TRUE = 5
JUMP_IF_FALSE = 6
LESS_THAN = 7
EQUALS = 8
HALT = 99

In [406]:
# Opcode param lengths
NUM_PARAMS = {
    ADD: 3,
    MULTIPLY: 3,
    INPUT: 1,
    OUTPUT: 1,
    JUMP_IF_TRUE: 2,
    JUMP_IF_FALSE: 2,
    LESS_THAN: 3,
    EQUALS: 3,
    HALT: 0,
}

In [407]:
# Opcode param modes
POSITION_MODE = 0
IMMEDIATE_MODE = 1

In [408]:
class Instruction:
    def __init__(self, opcode, length, params, modes, memory):
        self.opcode = opcode
        self.length = length
        self.params = params
        self.modes = modes
        self.memory = memory
        
    def get_val(self, param_index):
        if self.modes[param_index] == IMMEDIATE_MODE:
            return self.params[param_index]
        elif self.modes[param_index] == POSITION_MODE: 
            return self.memory[self.params[param_index]]
    
    def put_val(self, param_address_index, value):
        self.memory[self.params[param_address_index]] = value
        
    def __str__(self):
        return "opcode: {}, length: {}, params: {}, modes: {}".format(self.opcode, self.length, self.params, self.modes)
        

In [409]:
class Add(Instruction):
    def call(self):
        val0 = self.get_val(0)
        val1 = self.get_val(1)
        self.put_val(2, val0 + val1)

In [410]:
class Multiply(Instruction):
    def call(self):
        val0 = self.get_val(0)
        val1 = self.get_val(1)
        self.put_val(2, val0 * val1)   

In [411]:
INPUT_VALUE = 1 #default
class Input(Instruction):
    def call(self):
        self.put_val(0, INPUT_VALUE)

In [412]:
class Output(Instruction):
    def call(self):
        val = self.get_val(0)
        print("Output:", val)

In [413]:
class JumpIfTrue(Instruction):
    def call(self):
        in_val = self.get_val(0)
        out_val = self.get_val(1)
        if in_val:
            return out_val

In [414]:
class JumpIfFalse(Instruction):
    def call(self):
        in_val = self.get_val(0)
        out_val = self.get_val(1)
        if not in_val:
            return out_val        

In [415]:
class LessThan(Instruction):
    def call(self):
        val0 = self.get_val(0)
        val1 = self.get_val(1)
        out_val = 1 if val0 < val1 else 0
        self.put_val(2, out_val)           

In [416]:
class Equals(Instruction):
    def call(self):
        val0 = self.get_val(0)
        val1 = self.get_val(1)
        out_val = 1 if val0 == val1 else 0
        self.put_val(2, out_val)

In [417]:
class Halt(Instruction):
    def call(self):
        pass

In [418]:
OPS = {
    ADD: Add,
    MULTIPLY: Multiply,
    INPUT: Input,
    OUTPUT: Output,
    JUMP_IF_TRUE: JumpIfTrue,
    JUMP_IF_FALSE: JumpIfFalse,
    LESS_THAN: LessThan,
    EQUALS: Equals,
    HALT: Halt,
}

In [419]:
def create_instruction(memory, memory_pointer):
    encoded_opcode = memory[memory_pointer]
    opcode = int(str(encoded_opcode)[-2:])
    
    num_params = NUM_PARAMS[opcode]
    
    reverse_modes = str(encoded_opcode)[:-2][::-1]
    modes = [int(char) for char in reverse_modes] + [0] * (num_params - len(reverse_modes))
    
    params = memory[memory_pointer + 1 : memory_pointer + num_params + 1]
    
    return OPS[opcode](opcode, num_params + 1, params, modes, memory)

In [420]:
class Program:
    def __init__(self, initial_memory):
        self.memory = initial_memory[:]
        self.memory_pointer = 0
        
    def run(self):
        while(1):
            instruction = create_instruction(self.memory, self.memory_pointer)
            if instruction.opcode == HALT:
                break
                
            jump_pointer = instruction.call()
            self.memory_pointer = jump_pointer or (self.memory_pointer + instruction.length)

In [423]:
INPUT_VALUE = 1
Program(data).run()

Output: 0
Output: 0
Output: 0
Output: 0
Output: 0
Output: 0
Output: 0
Output: 0
Output: 0
Output: 5074395


In [424]:
INPUT_VALUE = 5
Program(data).run()

Output: 8346937
